In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import numpy as np
import seaborn as sns

df = pd.read_csv('SARSCoV2Biobank-RDVSEQ_DATA_2023-08-18_1047.csv')
hosp = pd.read_csv('/Users/tlh4362/Desktop/RDV/redcap/hospitalizations_aug2023.csv')
nanopore = pd.read_csv('/Users/tlh4362/Desktop/RDV/COV_sequences_duplicated_removed.csv')
ct = pd.read_csv('/Users/tlh4362/Desktop/RDV/SARSCoV2Biobank-SequencingDataAndCT_DATA_2023-08-29_1606.csv')
clinicaldata = pd.read_csv('/Users/tlh4362/Desktop/RDV/formatted_clinicaldata.csv')
vax = pd.read_csv('/Users/tlh4362/Desktop/RDV/formatted_vax.csv')
npbal = pd.read_csv('/Users/tlh4362/Desktop/RDV/SARSCoV2Biobank-Sampletype_DATA_LABELS_2023-12-11_1049.csv')
rdvdf = pd.read_csv('/Users/tlh4362/Desktop/RDV/updated_rdv.csv')


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (64,72,74,76,78,80,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
ct['ct_n1'] = ct['ct_n1'].replace({'na':None, 'undetermined':None}).astype(float)

In [3]:
df = pd.merge(df,  ct.loc[:,['record_id','ct_n1']], on = 'record_id', how = 'left')

In [4]:
#dropping all nanopore sequences
nanopore_idx = df[df['record_id'].isin(nanopore[nanopore['Illumina/Nanopore'] == 'Nanopore']['record_id'].astype(int))].index
print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))
df = df.drop(nanopore_idx, axis = 0)

print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))#dropping low quality sequences and 7274 was mislabeled UNKNOWN instead of nanopore

idx = df[df['record_id'].isin(pd.Series([11641, 11101, 7274]))].index #11641 and 11101 are low quality, 7274 is a nanopore

df = df.drop(idx, axis = 0)
print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))

(43855, 23)
6426
38011
(43614, 23)
6185
37823
(43611, 23)
6182
37823


In [5]:
#format
df['study_id'] = ['COV{}'.format(str(x).zfill(5)) for x in df['record_id']]

In [6]:
df = df[df['gisaid_id'].notnull()] #initial filter for only those with sequences

In [7]:
#correcting hospitalization dates
hosp['ca_hosp_admit_dt'] = hosp['admit_dt']

for idx in df[df['ca_hosp_admit_dt'].isnull()].index:
    rid = df.loc[idx,'record_id']
    if str(df.loc[idx,'ca_hosp_admit_dt']) == 'nan':
        daysdict = {}
        if len(hosp[hosp['record_id']==rid]['admit_dt'].unique()) > 1:
            print(rid)
            print(df[df['record_id']==rid]['mrn'].iloc[0])
            sample_date = pd.to_datetime(df[df['record_id']==rid]['sample_date'].sort_values().iloc[0])
            for hospdate in hosp[hosp['record_id']==rid]['admit_dt'].unique():
                hospdate = pd.to_datetime(hospdate)
                daysdict[hospdate] = abs((hospdate-sample_date).days)
            closesthosp = min(daysdict, key = daysdict.get)
            df.loc[idx,'ca_hosp_admit_dt'] = closesthosp
            df.loc[idx,'ca_hosp_admit'] = 1
        elif len(hosp[hosp['record_id']==rid]['admit_dt'].unique()) == 1:
            df.loc[idx,'ca_hosp_admit_dt'] = hosp[hosp['record_id']==rid]['admit_dt'].iloc[0]
            df.loc[idx,'ca_hosp_admit'] = 1

40774
519677
39716
690150
40770
849599
13795
1006310
19332
1487858
41598
1684739
41924
5077926
38197
5681532
40460
6536060
41970
6873081
41987
6873081
40936
6963571
13272
7440501
1399
7459602
41739
7487539
40786
7494924
27355
7500014
39760
7642679
41008
7856712
41324
7927569
41227
7930063
23038
7936434
39848
7970838
40052
7970838
41080
7995308
41555
8004514
41991
8004514
42664
8086371
40990
8256442
41724
8256442
17160
8470759
41734
8649925
40792
8787240
29329
8814087
40150
8836579
40769
9033847
12258
9115897
23371
9138519
1561
9183719
41074
9246940
10208
9285802
13334
9286273
40213
9298221
580
10326008
41169
11015434
34326
11122687


In [8]:
#collapsing remdesivir adminsitration dates
rdvdates = []
for idx in df.index:

    try:
        math.isnan(df.loc[idx,'ca_init_edw_remdes_ct_dt'])
        rdvdates.append(df.loc[idx,'ca_init_edw_remdes_ea_dt'])
    except:
        rdvdates.append(df.loc[idx,'ca_init_edw_remdes_ct_dt'])
#         print(df.loc[idx,['ca_init_edw_remdes_ct_dt','ca_init_edw_remdes_ea_dt']])

df['rdvdate'] = pd.to_datetime(rdvdates)
pd.to_datetime(df['rdvdate'].dropna()).sort_values()

12318   2020-03-13
13838   2020-03-15
9027    2020-03-21
9029    2020-03-21
25295   2020-03-23
           ...    
17389   2022-07-10
20653   2022-07-12
13161   2022-07-21
13162   2022-07-21
15563   2022-12-28
Name: rdvdate, Length: 578, dtype: datetime64[ns]

In [9]:
prenewrdv_mrn = df[df['rdvdate'].notnull()]['mrn']

In [10]:
#correcting for missing remdesivir administration date
for mrn in df['mrn'].unique():
    idx = df[df['mrn']==mrn].index
    if df[df['mrn']==mrn]['rdvdate'].isnull().sum() > 0:
        if mrn in rdvdf['mrn'].unique():
            rdvdate = pd.to_datetime(rdvdf[rdvdf['mrn']==mrn]['order_start_datetime']).sort_values().iloc[0]
            df.at[idx,'rdvdate'] = rdvdate
df['rdvdate'] = pd.to_datetime(df['rdvdate'])
df['rdvdate'].dropna().sort_values()

<ipython-input-10-4e12fa7685e2>:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mrn in rdvdf['mrn'].unique():


12318   2020-03-13
13838   2020-03-15
9029    2020-03-21
9027    2020-03-21
25295   2020-03-23
           ...    
9375    2023-03-01
15733   2023-03-05
14606   2023-03-14
14609   2023-03-14
3987    2023-11-28
Name: rdvdate, Length: 631, dtype: datetime64[ns]

In [11]:
#filtering out samples with missing sample collection dates, sequence data, hospitalization, and samples collected after 2020-01-01
df['sample_date'] = pd.to_datetime(df['sample_date'])
df['ca_hosp_admit_dt'] = pd.to_datetime(df['ca_hosp_admit_dt'])

print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))
df = df[df['sample_date'].notnull()] 

print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))

df = df[df['ca_hosp_admit'].notnull()] 
print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))

df = df[(df['sample_date'] >= pd.to_datetime('2020-01-01'))] #n = 5, samples before 2020, #& (df['sample_date'] <= pd.to_datetime('2022-08-01'))
print(df.shape)
print(df['gisaid_id'].notnull().sum())
print(len(df['mrn'].unique()))


#check dates from prevoius dataset

(6182, 25)
6182
5707
(6182, 25)
6182
5707
(4376, 25)
4376
3949
(4376, 25)
4376
3949


In [12]:
#filtering only for hospitalized patients
hosp = df[df['ca_hosp_admit']==1]

In [13]:
print(hosp.shape)
print(hosp['gisaid_id'].notnull().sum())
print(len(hosp['mrn'].unique()))

(1150, 25)
1150
792


In [14]:
#some patients who had RDV had samples that weren't associated with the RDV date. 

nordvidx = []
rdvidx = []
for mrn in hosp['mrn'].unique():
    #patients that had RDV but have at least one sample missing the associated RDV date so I impute the RDV date
    if hosp[(hosp['mrn']==mrn)]['rdvdate'].isnull().sum() != hosp[(hosp['mrn']==mrn)].shape[0] and hosp[(hosp['mrn']==mrn)]['rdvdate'].isnull().sum() != 0:      
        rdvdate = hosp[(hosp['mrn']==mrn)]['rdvdate'].dropna().iloc[0]
        idx = hosp[(hosp['mrn']==mrn) & (hosp['rdvdate'].isnull())].index
        hosp.at[idx,'rdvdate'] = rdvdate
    #then I check whether or not each patient has RDV or no RDV and get the index
    if hosp[(hosp['mrn']==mrn)]['rdvdate'].isnull().sum() == hosp[(hosp['mrn']==mrn)].shape[0]: #nordv
        nordvidx.extend(list(hosp[(hosp['mrn']==mrn)].index))
    elif hosp[(hosp['mrn']==mrn)]['rdvdate'].isnull().sum() == 0: #rdv
        rdvidx.extend(list(hosp[(hosp['mrn']==mrn)].index))


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
#days calculated all relative to first hosp or first RDV date
for mrn in hosp['mrn'].unique():
    pt = hosp[hosp['mrn']==mrn]
    hospdate = pt.sort_values(by = 'ca_hosp_admit_dt')['ca_hosp_admit_dt'].iloc[0] #first hosp date
    rdvdate = pt.sort_values(by = 'rdvdate')['rdvdate'].iloc[0] #first rdv date
    print(hospdate, rdvdate)
    daysdiffhosp = [x.days for x in (pt['sample_date'] - hospdate)]
    hosp.at[pt.index, 'daysdiff_sample_hosp'] = daysdiffhosp

    daysdiffrdvhosp = [x.days for x in (pt['rdvdate'] - hospdate)]
    hosp.at[pt.index, 'daysdiff_rdv_hosp'] = daysdiffrdvhosp

    daysdiffrdv = [x.days for x in (pt['sample_date'] - rdvdate)]
    hosp.at[pt.index, 'daysdiff_sample_rdv'] = daysdiffrdv

    

2022-05-29 00:00:00 2022-06-14 00:00:00
2022-12-16 00:00:00 2022-12-16 00:00:00
2020-03-30 00:00:00 NaT
2020-05-11 00:00:00 NaT
2022-11-04 00:00:00 NaT
2021-10-15 00:00:00 NaT
2021-05-04 00:00:00 2021-05-04 00:00:00
2021-10-17 00:00:00 2021-10-20 00:00:00
2022-12-15 00:00:00 NaT
2021-12-09 00:00:00 2021-12-09 00:00:00
2021-12-14 00:00:00 2021-12-15 00:00:00
2021-11-21 00:00:00 NaT
2021-05-12 00:00:00 2021-05-13 00:00:00
2021-01-12 00:00:00 2021-01-13 00:00:00
2021-10-12 00:00:00 NaT
2022-06-15 00:00:00 NaT
2021-12-06 00:00:00 2021-12-06 00:00:00
2022-03-16 00:00:00 2022-03-16 00:00:00
2020-12-18 00:00:00 2020-12-18 00:00:00
2020-11-10 00:00:00 2020-11-10 00:00:00
2022-01-02 00:00:00 2022-01-02 00:00:00
2021-08-25 00:00:00 2021-08-26 00:00:00
2020-12-15 00:00:00 2020-11-26 00:00:00
2022-01-06 00:00:00 2022-01-08 00:00:00
2022-05-18 00:00:00 2022-05-18 00:00:00
2022-01-20 00:00:00 2022-01-20 00:00:00
2022-03-28 00:00:00 2022-01-05 00:00:00
2020-10-20 00:00:00 NaT
2021-08-25 00:00:00 2021

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


 2021-03-28 00:00:00
2020-10-29 00:00:00 2020-10-29 00:00:00
2022-03-27 00:00:00 NaT
2021-11-30 00:00:00 2021-12-01 00:00:00
2022-11-09 00:00:00 NaT
2023-02-01 00:00:00 NaT
2021-12-02 00:00:00 2021-11-24 00:00:00
2022-04-25 00:00:00 NaT
2022-06-03 00:00:00 NaT
2022-09-25 00:00:00 NaT
2022-03-08 00:00:00 NaT
2020-07-17 00:00:00 NaT
2022-10-01 00:00:00 2022-10-02 00:00:00
2020-11-13 00:00:00 2020-11-14 00:00:00
2022-01-03 00:00:00 2022-01-10 00:00:00
2021-10-18 00:00:00 NaT
2021-09-12 00:00:00 2021-09-12 00:00:00
2022-03-08 00:00:00 NaT
2021-07-02 00:00:00 2021-07-02 00:00:00
2020-11-20 00:00:00 2020-11-21 00:00:00
2022-10-28 00:00:00 NaT
2020-03-14 00:00:00 NaT
2022-06-27 00:00:00 2022-06-28 00:00:00
2020-12-20 00:00:00 2020-12-20 00:00:00
2020-11-16 00:00:00 2020-11-16 00:00:00
2021-11-02 00:00:00 2021-11-02 00:00:00
2022-01-07 00:00:00 2021-12-24 00:00:00
2020-12-29 00:00:00 NaT
2020-08-14 00:00:00 2020-08-17 00:00:00
2022-03-30 00:00:00 2021-12-10 00:00:00
2022-01-03 00:00:00 2022-01

2022-12-04 00:00:00 NaT
2020-11-23 00:00:00 2020-12-04 00:00:00
2022-12-11 00:00:00 2022-12-11 00:00:00
2021-12-27 00:00:00 2021-12-28 00:00:00
2023-02-20 00:00:00 2023-02-08 00:00:00
2022-01-20 00:00:00 2022-01-20 00:00:00
2023-03-03 00:00:00 2023-03-05 00:00:00
2020-04-09 00:00:00 NaT
2022-05-17 00:00:00 NaT
2021-01-07 00:00:00 NaT
2020-05-25 00:00:00 2020-05-26 00:00:00
2021-12-18 00:00:00 2022-01-03 00:00:00
2021-12-29 00:00:00 2021-12-29 00:00:00
2021-11-10 00:00:00 2021-11-10 00:00:00
2022-05-06 00:00:00 NaT
2022-01-14 00:00:00 2022-01-15 00:00:00
2023-01-18 00:00:00 NaT
2021-01-28 00:00:00 2021-01-29 00:00:00
2022-05-12 00:00:00 NaT
2022-10-21 00:00:00 NaT
2021-12-04 00:00:00 2021-12-04 00:00:00
2022-06-28 00:00:00 NaT
2020-04-27 00:00:00 NaT
2020-04-29 00:00:00 NaT
2022-01-21 00:00:00 2020-11-22 00:00:00
2020-04-01 00:00:00 NaT
2020-08-03 00:00:00 2020-08-03 00:00:00
2020-04-05 00:00:00 NaT
2020-11-20 00:00:00 2020-11-24 00:00:00
2021-02-28 00:00:00 2021-02-28 00:00:00
2022-02-

2020-05-07 00:00:00 NaT
2021-05-05 00:00:00 2021-05-07 00:00:00
2021-02-24 00:00:00 NaT
2022-08-13 00:00:00 NaT
2022-04-08 00:00:00 2022-04-25 00:00:00
2020-12-03 00:00:00 2020-12-04 00:00:00
2021-12-08 00:00:00 NaT
2021-01-12 00:00:00 2021-01-13 00:00:00
2022-01-28 00:00:00 2022-01-29 00:00:00
2020-04-14 00:00:00 NaT
2020-04-27 00:00:00 NaT
2021-02-13 00:00:00 NaT
2020-10-30 00:00:00 2020-10-31 00:00:00
2022-02-16 00:00:00 2022-02-17 00:00:00
2021-09-29 00:00:00 2021-09-29 00:00:00
2021-02-16 00:00:00 2021-02-19 00:00:00
2021-01-13 00:00:00 2021-01-13 00:00:00
2020-12-02 00:00:00 2020-12-02 00:00:00
2020-12-23 00:00:00 2020-12-24 00:00:00
2021-05-10 00:00:00 2021-04-16 00:00:00
2021-12-29 00:00:00 2021-12-29 00:00:00
2020-12-08 00:00:00 2020-12-09 00:00:00
2020-04-15 00:00:00 NaT
2020-12-02 00:00:00 2020-12-02 00:00:00
2022-01-27 00:00:00 2022-01-27 00:00:00
2022-12-21 00:00:00 NaT
2021-08-04 00:00:00 2021-08-04 00:00:00
2020-04-12 00:00:00 NaT
2021-04-23 00:00:00 2021-04-23 00:00:00


In [16]:
# label = []
# for idx in hosp.index:
#     sampledate = hosp.loc[idx,'sample_date']
#     rdvdate = hosp.loc[idx,'rdvdate']
#     if str(rdvdate) == 'NaT':
#         label.append('no_rdv')
#     elif sampledate <= rdvdate:
#         label.append('before_rdv')
#     else:
#         label.append('after_rdv')
# hosp['sample_label'] = label
# hosp[hosp['sample_label']=='after_rdv'].loc[:,['sample_date','rdvdate']]

In [17]:
nordv_hosp = hosp.loc[nordvidx,:]
rdv_hosp = hosp.loc[rdvidx,:]

In [18]:
#labeling samples according to their days between sampling and remdesivir administration or hospitalization
dflist = [rdv_hosp,nordv_hosp]
nameslist = ['rdv','nordv']

for d in range(len(nameslist)):
    df = dflist[d]
    name = nameslist[d]
    if name == 'rdv':
        daydiff = 'daysdiff_sample_rdv'
    else:
        daydiff = 'daysdiff_sample_hosp'
    for mrn in df['mrn'].unique():
        pt = df[df['mrn']==mrn]
        idx = pt.index
        if pt.shape[0] == 1: #single specimen
            if pt[daydiff].iloc[0] <= 0 and pt[daydiff].iloc[0] >= -60: #single specimen before or on day of hospitalization
                df.at[idx, 'sample_type'] = 'toss'
            elif pt[daydiff].iloc[0] >= 1 and pt[daydiff].iloc[0] <= 90: #single specimens day after hospitalization
                df.at[idx, 'sample_type'] = 'one_sample_post'


        else:
            if any(x >= -60 and x <= 0 for x in pt[daydiff]) and any(x >= 1 and x <= 90 for x in pt[daydiff]): #multiple specimens that have at least one pair
                if pt.shape[0] == 2:
                    df.at[idx, 'sample_type'] = 'two_sample_paired'
                elif pt.shape[0] > 2:
                    if (pt.loc[:,[daydiff]]>=1).sum()[0] == 1: #the patient has only 1 post sample, so it would be classified as two sample paired
                        df.at[pt[pt[daydiff]>=1].index, 'sample_type'] ='two_sample_paired' #labeling post sample first


                        closesttordv = abs(pt[daydiff]).min() #finding the pre sample closest to RDV
                        if pt[pt[daydiff]==closesttordv].shape[0] == 1: #one sample closest to RDV
                            df.at[pt[pt[daydiff]==closesttordv].index,'sample_type'] = 'two_sample_paired'
                        elif pt[pt[daydiff]==closesttordv].shape[0] == 2: #multiple samples collected same day as RDV (daysdiff = 0)
                            df.at[pt[pt[daydiff]==closesttordv].index[0],'sample_type'] = 'two_sample_paired'
                        else:
                            df.at[pt[pt[daydiff]==(closesttordv)*-1].index[0], 'sample_type'] = 'two_sample_paired' #multiple samples collected on the same day but before RDV (daysdiff < 0)
                    else:
                        if len(pt[pt[daydiff] >= 1][daydiff].unique()) == 1: #post samples are all on the same day, will label as two sample paired
                            df.at[pt[pt[daydiff] >= 1].index[0], 'sample_type'] = 'two_sample_paired'
                            df.at[pt[pt[daydiff] <= 0].index[0], 'sample_type'] = 'two_sample_paired'
                        else:
                            df.at[idx, 'sample_type'] = 'three_plus_paired'
            elif all(x>=1 for x in pt[daydiff]): #multiple specimens all post
                if len(pt[pt[daydiff] >= 1][daydiff].unique()) == 1:
                    df.at[pt[pt[daydiff] >= 1].index[0], 'sample_type'] = 'one_sample_post'
                else:
                    df.at[idx, 'sample_type'] = 'multiple_sample_post'

            elif all(x<=0 for x in pt[daydiff]): #multiple specimens all pre
                df.at[idx, 'sample_type'] = 'toss'


    #manual editing
    idx = df[df['sample_type'].isnull()].index
    df.at[idx,'sample_type'] = 'toss' #outside dateranges
    df['mrn'] = df['mrn'].astype(int)

In [19]:
#dropping samples with multiple "pres"
for mrn in rdv_hosp['mrn'].unique():
    pt = rdv_hosp[rdv_hosp['mrn']==mrn]
    if pt[pt['daysdiff_sample_rdv']<=0].shape[0] > 1 and 'toss' not in pt['sample_type'].unique():
        print(pt.loc[:,['sample_date','rdvdate','daysdiff_sample_rdv','sample_type']]) #these patients had multiple pre samples

rdv_hosp = rdv_hosp.drop([9744, 13504, 28774], axis = 0)
#mrn 7500014, 7908664 and 10575022 had multiple pre samples

     sample_date    rdvdate  daysdiff_sample_rdv        sample_type
9744  2022-01-20 2022-01-20                  0.0  three_plus_paired
9745  2022-01-20 2022-01-20                  0.0  three_plus_paired
9746  2022-02-28 2022-01-20                 39.0  three_plus_paired
9747  2022-03-31 2022-01-20                 70.0  three_plus_paired
9748  2022-04-06 2022-01-20                 76.0  three_plus_paired
9749  2022-05-02 2022-01-20                102.0  three_plus_paired
9750  2022-05-17 2022-01-20                117.0  three_plus_paired
      sample_date    rdvdate  daysdiff_sample_rdv        sample_type
13504  2021-12-24 2021-12-24                  0.0  three_plus_paired
13505  2021-12-24 2021-12-24                  0.0  three_plus_paired
13506  2022-01-13 2021-12-24                 20.0  three_plus_paired
13508  2022-01-18 2021-12-24                 25.0  three_plus_paired
13509  2022-01-26 2021-12-24                 33.0  three_plus_paired
13515  2022-02-27 2021-12-24              

In [20]:
#dropping samples with multiple "pres"

for mrn in nordv_hosp['mrn'].unique():
    pt = nordv_hosp[nordv_hosp['mrn']==mrn]
    if pt[pt['daysdiff_sample_hosp']<=0].shape[0] > 1  and 'toss' not in pt['sample_type'].unique():
        print(pt.loc[:,['sample_date','ca_hosp_admit_dt','daysdiff_sample_hosp','sample_type']]) #these patients had multiple pre samples
nordv_hosp = nordv_hosp.drop([35887], axis = 0)
#mrn 111012976155 had multiple pres



      sample_date ca_hosp_admit_dt  daysdiff_sample_hosp        sample_type
35887  2020-03-24       2020-03-25                  -1.0  three_plus_paired
35888  2020-03-25       2020-03-25                   0.0  three_plus_paired
35892  2020-03-27       2020-03-25                   2.0  three_plus_paired
35893  2020-04-04       2020-03-26                  10.0  three_plus_paired
35895  2020-03-27       2020-03-26                   2.0  three_plus_paired


In [21]:
#manual correction of samples with unique conditions (i.e. multiple hospitalization dates, or multiple sample dates or both)
for mrn in nordv_hosp[nordv_hosp['daysdiff_sample_hosp']<-60]['mrn'].unique():
    if 'toss' not in nordv_hosp[nordv_hosp['mrn']==mrn]['sample_type'].unique():
        print(nordv_hosp[nordv_hosp['mrn']==mrn].loc[:,['record_id', 'mrn', 'sample_date','ca_hosp_admit_dt','daysdiff_sample_hosp','sample_type']])
    # mrn 8461739 has two samples with two hospital admit dates, if I use the first hosp admit date, the second sample has to be dropped and the patient changes frfom a multisample post to a one sample post. 
    
print(nordv_hosp[nordv_hosp['mrn']==8461739].loc[:,['record_id', 'mrn', 'sample_date','ca_hosp_admit_dt','daysdiff_sample_hosp','sample_type']])
nordv_hosp = nordv_hosp.drop(19191, axis = 0) #pt with two samples (2 sample dates) and two hosp dates, dropping second sample

nordv_hosp.at[19190,'sample_type'] = 'one_sample_post'
print(nordv_hosp[nordv_hosp['mrn']==8461739].loc[:,['record_id', 'mrn', 'sample_date','ca_hosp_admit_dt','daysdiff_sample_hosp','sample_type']])





       record_id      mrn sample_date ca_hosp_admit_dt  daysdiff_sample_hosp  \
19190      25018  8461739  2022-03-09       2022-02-26                  11.0   
19191      32562  8461739  2022-06-12       2022-06-08                 106.0   

                sample_type  
19190  multiple_sample_post  
19191  multiple_sample_post  
       record_id      mrn sample_date ca_hosp_admit_dt  daysdiff_sample_hosp  \
19190      25018  8461739  2022-03-09       2022-02-26                  11.0   

           sample_type  
19190  one_sample_post  


In [22]:
print(rdv_hosp[rdv_hosp['mrn']==6710573].loc[:,['record_id', 'mrn', 'sample_date','rdvdate','daysdiff_sample_rdv','sample_type']])
rdv_hosp = rdv_hosp.drop(5978, axis = 0) #pt with two samples (2 sample dates) and two hosp dates, dropping second sample

rdv_hosp.at[5976,'sample_type'] = 'one_sample_post'
print(rdv_hosp[rdv_hosp['mrn']==6710573].loc[:,['record_id', 'mrn', 'sample_date','rdvdate','daysdiff_sample_rdv','sample_type']])


      record_id      mrn sample_date    rdvdate  daysdiff_sample_rdv  \
5976      13051  6710573  2021-09-28 2021-08-27                 32.0   
5978      31685  6710573  2022-05-31 2022-06-01                277.0   

               sample_type  
5976  multiple_sample_post  
5978  multiple_sample_post  
      record_id      mrn sample_date    rdvdate  daysdiff_sample_rdv  \
5976      13051  6710573  2021-09-28 2021-08-27                 32.0   

          sample_type  
5976  one_sample_post  


In [23]:
for mrn in rdv_hosp[rdv_hosp['daysdiff_sample_rdv']>90]['mrn'].unique():
    if rdv_hosp[rdv_hosp['mrn']==mrn]['sample_type'].unique()[0] != 'toss':
        print(rdv_hosp[rdv_hosp['mrn']==mrn].loc[:,['record_id','mrn', 'sample_date','rdvdate','daysdiff_sample_rdv','sample_type']])

rdv_hosp = rdv_hosp.drop([1538, 9749, 9750,10679 ], axis = 0) # mrn 5072963, 7500014, and 7609797 are labeled multisample paired, but all have at least one sample past the 90 day range. Should not change anything but we lose 4 sequences. 
rdv_hosp = rdv_hosp.drop([17351, 17353, 17354], axis = 0) #mrn 8256442 is labeled multisample post but all are beyond 90 days




      record_id      mrn sample_date    rdvdate  daysdiff_sample_rdv  \
1534       6680  5072963  2021-02-11 2021-01-20                 75.0   
1535       6768  5072963  2021-02-17 2021-01-20                 81.0   
1536       6794  5072963  2021-02-17 2021-01-20                 81.0   
1537       6893  5072963  2021-02-24 2021-01-20                 88.0   
1538       7133  5072963  2021-03-16 2021-01-20                108.0   
1539      11662  5072963  2020-11-28 2020-11-28                  0.0   
1542      12138  5072963  2021-01-04 2021-11-28                 37.0   
1543      12223  5072963  2021-01-06 2020-11-28                 39.0   
1544      12384  5072963  2021-01-26 2021-01-20                 59.0   
1545      12414  5072963  2021-01-19 2021-01-20                 52.0   

            sample_type  
1534  three_plus_paired  
1535  three_plus_paired  
1536  three_plus_paired  
1537  three_plus_paired  
1538  three_plus_paired  
1539  three_plus_paired  
1542  three_plus_paired  

In [24]:
rdv_hosp = rdv_hosp[rdv_hosp['sample_type'] != 'toss']
nordv_hosp = nordv_hosp[nordv_hosp['sample_type'] != 'toss']

# rdv_hosp = rdv_hosp[(rdv_hosp['daysdiff_sample_rdv']<=90) & (rdv_hosp['daysdiff_sample_rdv']>-60)]
# nordv_hosp = nordv_hosp[(nordv_hosp['daysdiff_sample_hosp']<=90) & (nordv_hosp['daysdiff_sample_hosp']>-60)]



In [25]:
#manual correction of typos
idx = rdv_hosp[rdv_hosp['mrn']==7465848].index

rdv_hosp.at[idx, 'ca_hosp_admit_dt']= pd.to_datetime('2020-07-17')
rdv_hosp.at[idx, 'daysdiff_sample_hosp']= -10
rdv_hosp.at[idx, 'daysdiff_rdv_hosp']= -11


idx = rdv_hosp[rdv_hosp['mrn']==5072963].index

rdv_hosp.at[idx,'rdvdate'] = pd.to_datetime('2020-11-28')
rdv_hosp.at[idx,'daysdiff_rdv_hosp'] = -9


idx = rdv_hosp[rdv_hosp['mrn']==8083659].index

rdv_hosp.at[idx,'rdvdate'] = pd.to_datetime('2021-12-28')
rdv_hosp.at[idx,'daysdiff_rdv_hosp'] = 1

In [26]:
for label in rdv_hosp['sample_type'].unique():
    print('###', label)
    print('unique mrn', len(rdv_hosp[rdv_hosp['sample_type']==label]['mrn'].unique()))
    print('unique samples', rdv_hosp[rdv_hosp['sample_type']==label].shape[0])
#     rdv_hosp[rdv_hosp['sample_type']==label].loc[:,['study_id','mrn','sample_date','ca_hosp_admit_dt','rdvdate']].to_csv('/Users/tlh4362/Desktop/rdv_{}.csv'.format(label))
#     7029 is a one sample post so we lost that
#     11101 and 11641 both belonged to the same patient and would have been multiple sample post so lost one patient and two samples
#     98, 142 and 672 are from 3 different patients that all had another sequence from the same sample so didn't lose those

print('############################')
for label in nordv_hosp['sample_type'].unique():
    print('###',label)
    print('unique mrn', len(nordv_hosp[nordv_hosp['sample_type']==label]['mrn'].unique()))
    print('unique samples', nordv_hosp[nordv_hosp['sample_type']==label].shape[0])
#     nordv_hosp[nordv_hosp['sample_type']==label].loc[:,['study_id','mrn','sample_date','ca_hosp_admit_dt','rdvdate']].to_csv('/Users/tlh4362/Desktop/control_{}.csv'.format(label))


### multiple_sample_post
unique mrn 40
unique samples 103
### one_sample_post
unique mrn 98
unique samples 98
### two_sample_paired
unique mrn 34
unique samples 68
### three_plus_paired
unique mrn 20
unique samples 94
############################
### one_sample_post
unique mrn 125
unique samples 125
### multiple_sample_post
unique mrn 42
unique samples 122
### three_plus_paired
unique mrn 10
unique samples 38
### two_sample_paired
unique mrn 16
unique samples 32


In [27]:
npbal['Sample Type (when received) '] = npbal['Sample Type (when received) '].replace({'RNA':None}) #if its RNA we don't know so just treat it as missing
npbal['record_id'] = npbal['Record ID']
rdv_hosp = pd.merge(rdv_hosp, npbal, on = 'record_id', how = 'left')
nordv_hosp = pd.merge(nordv_hosp, npbal, on = 'record_id', how = 'left')

In [28]:
#reconciling samples by sample type (i.e. NP vs BAL)
count = 0
for mrn in rdv_hosp['mrn'].unique():
    pt = rdv_hosp[rdv_hosp['mrn']==mrn]
    if len(pt['Sample Type (when received) '].unique()) > 1:
        if pt['Sample Type (when received) '].isnull().sum()==0: #patients with multiple sample types and non missing labels
            print('mrn = {}'.format(mrn))
            dominant_sampletype = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).index[0]
            minority_sampletype = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).index[1]
            dominant_count = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).values[0]
            minority_count = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).values[1]
            if dominant_count == minority_count:
                if all(pt['daysdiff_sample_rdv'] > 0): #all post
                    todrop_idx = pt[pt['Sample Type (when received) ']=='BAL'].index
                    rdv_hosp = rdv_hosp.drop(todrop_idx, axis = 0)
                    pt = rdv_hosp[rdv_hosp['mrn']==mrn]
                    if pt.shape[0] == 1:
                        rdv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'
                    else:
                        rdv_hosp.at[pt.index, 'sample_type'] = 'multiple_sample_post'
                else:
                    if pt.shape[0] == 2:
                        todrop_idx = pt['daysdiff_sample_rdv'].sort_values().index[0]
                        rdv_hosp = rdv_hosp.drop(todrop_idx, axis = 0)
                        pt = rdv_hosp[rdv_hosp['mrn']==mrn]
                        rdv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'

                    else:
                        print(pt.loc[:,['record_id', 'sample_type', 'Sample Type (when received) ', 'rdvdate','sample_date', 'daysdiff_sample_rdv']])
                        #manual change below
            else:
                todrop_idx = pt[pt['Sample Type (when received) ']==minority_sampletype].index
                rdv_hosp = rdv_hosp.drop(todrop_idx, axis = 0)
                pt = rdv_hosp[rdv_hosp['mrn']==mrn]
               
                if all(pt['daysdiff_sample_rdv'] > 0): 
                    if pt.shape[0] == 1:
                        rdv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'
                    else:
                        rdv_hosp.at[pt.index, 'sample_type'] = 'multiple_sample_post'
                else:
                    if pt.shape[0] == 2:
                        rdv_hosp.at[pt.index, 'sample_type'] = 'two_sample_paired'
                    else:  
                        rdv_hosp.at[pt.index, 'sample_type'] = 'three_plus_paired'
                
            
            
rid_list = [10641, 32688, 32647, 32785, 1208, 32750]
for rid in rid_list:
    idx = rdv_hosp[rdv_hosp['record_id']==rid].index
    rdv_hosp = rdv_hosp.drop(idx, axis = 0)


rdv_hosp.at[[80,81],'sample_type'] = 'two_sample_paired' #mrn = 6835837
rdv_hosp.at[[183,184],'sample_type'] = 'two_sample_paired' #mrn = 7936279
rdv_hosp.at[255,'sample_type'] = 'one_sample_post' #mrn = 8700862



'''
mrn = 1383162 had 4 samples, classified as multiple sample post. 3 of the samples were NP with 1 sample (record id = 27826) being a BAL so we dropped it
mrn = 5372736 had 4 samples, classified as multiple sample post. 3 of the samples were NP with 1 sample (record id = 13050) being a BAL so we dropped it
mrn = 5449657 had 3 samples, classified as three plus paired. 2 of the samples were BAL with 1 sample (record id = 11470) being a NP so we dropped it, reclassified the patient to multiple samples post
mrn = 5825135 had 2 samples, classified as multiple sample post. 1 of the samples were NP with 1 sample (record id = 13049) being a BAL dropped it, reclassified the patient to one sample post
MANUAL CHANGE: mrn = 6835837 had 4 samples, classified as three plus paired. 2 of the samples were NP with 2 samples (record id = 10641,32688) being a BAL so we dropped it, reclassified the patient to two sample paired
mrn = 7394063 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record_id = 32780) being a BAL, so we dropped it
mrn = 7666146 had 3 samples, classified as three sample paired. 2 of the samples was NP with 1 sample (record_id = 6618) being a RNA, so we dropped it and reclassified to two sample paired
mrn = 7752664 had 2 samples, classified as two sample paired. 1 of the samples was NP with 1 sample (record_id = 5399) being a RNA, so we dropped it and reclassified to one sample post
MANUAL CHANGE: mrn = 7936279 had 4 samples, classified as three plus paired. 2 of the samples was NP with 2 samples (record_id = 32647,32785) being a BAL, so we dropped it and reclassified to two sample paired
mrn = 8556968 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record_id = 10605) being a BAL, so we dropped it and reclassified to one sample post
MANUAL CHANGE:mrn = 8700862 had 3 samples, classified as three plus paired. 1 of the samples was NP with 2 samples (record_id = 1208 and 32750) being a RNA and BAL, so we dropped those and reclassified to one sample post
mrn = 8854391 had 2 samples, classified as two sample paired. 1 of the samples was NP with 1 sample (record_id = 13060) being a BAL, so we dropped it and reclassified to one sample post
mrn = 8959156 had 2 samples, classified as two sample paired. 1 of the samples was BAL with 1 sample (record_id = 8406) being an NP, so we dropped the NP because it was the "pre" sample and reclassified to one sample post
mrn = 9065253 had 2 samples, classified as two sample paired. 1 of the samples was NP with 1 sample (record_id = 12506) being a BAL, so we dropped it and reclassified to one sample post
mrn = 9200960 had 4 samples, classified as three plus paired. 3 of the samples was BAL with 1 sample (record_id = 10203) being a NP, so we dropped it
mrn = 9249352 had 2 samples, classified as two sample paired. 1 of the samples was NP with 1 sample (record_id = 137) being a RNA, so we dropped it and reclassified to one sample post
mrn = 111013115428 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record_id = 32764) being a BAL, so we dropped it and reclassified to one sample post

'''
            

mrn = 1383162
mrn = 5372736
mrn = 5449657
mrn = 5825135
mrn = 6835837
    record_id        sample_type Sample Type (when received)     rdvdate  \
79      10641  three_plus_paired                          BAL 2020-05-19   
80      11220  three_plus_paired                     VTM (NP) 2020-05-19   
81      11316  three_plus_paired                     VTM (NP) 2020-05-19   
82      32688  three_plus_paired                          BAL 2020-05-19   

   sample_date  daysdiff_sample_rdv  
79  2020-05-28                  9.0  
80  2020-05-18                 -1.0  
81  2020-05-27                  8.0  
82  2020-06-06                 18.0  
mrn = 7394063
mrn = 7936279
     record_id        sample_type Sample Type (when received)     rdvdate  \
183      12568  three_plus_paired                     VTM (NP) 2020-09-25   
184      12593  three_plus_paired                     VTM (NP) 2020-09-25   
185      32647  three_plus_paired                          BAL 2020-09-25   
186      32785  three_p

'\nmrn = 1383162 had 4 samples, classified as multiple sample post. 3 of the samples were NP with 1 sample (record id = 27826) being a BAL so we dropped it\nmrn = 5372736 had 4 samples, classified as multiple sample post. 3 of the samples were NP with 1 sample (record id = 13050) being a BAL so we dropped it\nmrn = 5449657 had 3 samples, classified as three plus paired. 2 of the samples were BAL with 1 sample (record id = 11470) being a NP so we dropped it, reclassified the patient to multiple samples post\nmrn = 5825135 had 2 samples, classified as multiple sample post. 1 of the samples were NP with 1 sample (record id = 13049) being a BAL dropped it, reclassified the patient to one sample post\nMANUAL CHANGE: mrn = 6835837 had 4 samples, classified as three plus paired. 2 of the samples were NP with 2 samples (record id = 10641,32688) being a BAL so we dropped it, reclassified the patient to two sample paired\nmrn = 7394063 had 2 samples, classified as multiple sample post. 1 of the 

In [29]:
#reconciling samples by sample type (i.e. NP vs BAL)

count = 0
for mrn in nordv_hosp['mrn'].unique():
    pt = nordv_hosp[nordv_hosp['mrn']==mrn]
    if len(pt['Sample Type (when received) '].unique()) > 1:
        if pt['Sample Type (when received) '].isnull().sum()==0: #patients with multiple sample types and non missing labels
            dominant_sampletype = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).index[0]
            minority_sampletype = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).index[1]
            dominant_count = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).values[0]
            minority_count = pt['Sample Type (when received) '].value_counts().sort_values(ascending = False).values[1]
            if dominant_count == minority_count:
                if all(pt['daysdiff_sample_hosp'] > 0): #all post

                    todrop_idx = pt[pt['Sample Type (when received) ']=='BAL'].index
                    nordv_hosp = nordv_hosp.drop(todrop_idx, axis = 0)
                    pt = nordv_hosp[nordv_hosp['mrn']==mrn]
                    if pt.shape[0] == 1:
                        nordv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'
                    else:
                        nordv_hosp.at[pt.index, 'sample_type'] = 'multiple_sample_post'
                else:
                    pass

#                     if pt.shape[0] == 2:
                
#                         todrop_idx = pt['daysdiff_sample_hosp'].sort_values().index[0]
#                         nordv_hosp = nordv_hosp.drop(todrop_idx, axis = 0)
#                         pt = nordv_hosp[nordv_hosp['mrn']==mrn]
#                         nordv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'

#                     else:
#                         pass
#                         print(pt.loc[:,['record_id', 'sample_type', 'Sample Type (when received) ', 'rdvdate','sample_date', 'daysdiff_sample_hosp']])
#                         #manual change below
            else:

                todrop_idx = pt[pt['Sample Type (when received) ']==minority_sampletype].index
                nordv_hosp = nordv_hosp.drop(todrop_idx, axis = 0)
                pt = nordv_hosp[nordv_hosp['mrn']==mrn]
               
                if all(pt['daysdiff_sample_hosp'] > 0): 
                    if pt.shape[0] == 1:
                        nordv_hosp.at[pt.index, 'sample_type'] = 'one_sample_post'
                    else:
                        nordv_hosp.at[pt.index, 'sample_type'] = 'multiple_sample_post'
                else:
                    if pt.shape[0] == 2:
                        nordv_hosp.at[pt.index, 'sample_type'] = 'two_sample_paired'
                    else:  
                        nordv_hosp.at[pt.index, 'sample_type'] = 'three_plus_paired'
# to_drop = [204,206,208]            
# nordv_hosp = nordv_hosp.drop(to_drop, axis = 0)



         
'''
mrn = 5281380 had 3 samples, classified as multiple sample post. 2 of the samples were BAL with 1 sample (record id = 8444) being an NP so we dropped it
mrn = 6351476 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record id = 8429) being an NP so we dropped it and reclassified it to one sample post
mrn = 6891929 had 3 samples, classified as multiple sample post. 2 of the samples was BAL with 1 sample (record id = 8474) being an NP so we dropped it
mrn = 7149759 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record id = 32689) being a BAL so we dropped it and reclassified it to one sample post
mrn = 7261558 had 2 samples, classified as two sample paired. 1 of the samples was RNA with 1 sample (record id = 8482) being a BAL so we dropped it and reclassified it to one sample post
mrn = 7435740 had 7 samples, classified as multiple sample post. 4 of the samples was BAL with 1 sample (record id = 10228) being a NP and 2 samples (record id = 184, 377) being an RNA so we dropped those
mrn = 7822933 had 7 samples, classified as multiple sample post. 4 of the samples was BAL with 3 samples (record id = 8443,11169,11206 ) being a NP so we dropped those
mrn = 8265225 had 4 samples, classified as multiple sample post. 2 of the samples was BAL with 2 samples (record id = 82,87) being a NP so we dropped those
mrn = 8278762 had 5 samples, classified as three plus paired. 4 of the samples was BAL with 1 samples (record id = 8401) being a NP so we dropped it and reclassified to multiple sample post
mrn = 8541230 had 3 samples, classified as three plus paired. 2 of the samples was BAL with 1 samples (record id = 10218) being a NP so we dropped it and reclassified to multiple sample post
mrn = 8618399 had 3 samples, classified as multiple sample post. 2 of the samples was BAL with 1 samples (record id = 11143) being a NP so we dropped it 
mrn = 111012032131 had 3 samples, classified as multiple sample post. 2 of the samples was NP with 1 samples (record id = 13043) being a BAL so we dropped it 
MANUAL CHANGE mrn = 111012411416 had 6 samples, classified as three plus paired. 3 of the samples was NP with 3 samples (record id = 8518,10953,32627) being BALs so we dropped it 
mrn = 111012976155 had 4 samples, classified as three plus paired. 2 of the samples was BAL with 1 sample (record id = 230) being RNA and 1 sample (record id = 8484) being NP so we dropped it and reclassified to multiple sample post
mrn = 111012983066 had 2 samples, classified as two sample paired. 1 of the samples was BAL with 1 sample (record id = 11071) being NP so we dropped it and reclassified to one sample post
mrn = 111012986194 had 4 samples, classified as three plus paired. 3 of the samples was BAL with 1 sample (record id = 8473) being NP so we dropped it and reclassified to multiple sample post
mrn = 111012998802 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record id = 32666) being BAL so we dropped it and reclassified to one sample post
mrn = 111012999279 had 3 samples, classified as multiple sample post. 2 of the samples was BAL with 1 samples (record id = 8450) being a NP so we dropped it 
mrn = 111013000962 had 4 samples, classified as three plus paired. 3 of the samples was NP with 1 sample (record id = 32670) being BAL so we dropped it
mrn = 111013020039 had 5 samples, classified as multiple sample post. 3 of the samples was BAL with 2 samples (record id = 8442, 11365) being NP so we dropped it
mrn = 111013070579 had 4 samples, classified as multiple sample post. 3 of the samples was BAL with 1 sample (record id = 11392) being NP so we dropped it
mrn = 111013254660 had 4 samples, classified as multiple sample post. 3 of the samples was RNA with 1 sample (record id = 6940) being NP so we dropped it
mrn = 111013576734 had 3 samples, classified as multiple sample post. 2 of the samples was NP with 1 sample (record id = 13059) being BAL so we dropped it
mrn = 111014005639 had 3 samples, classified as multiple sample post. 2 of the samples was BAL with 1 sample (record id = 7025) being NP so we dropped it
mrn = 111014014202 had 4 samples, classified as multiple sample post. 2 of the samples was NP with 2 samples (record id = 10611, 10620) being BAL so we dropped it

mrn = 111014291619 had 3 samples, classified as three plus paired. 2 of the samples was NP with 1 samples (record id = 13053) being BAL so we dropped it and reclassified to two sample paired


'''            



'\nmrn = 5281380 had 3 samples, classified as multiple sample post. 2 of the samples were BAL with 1 sample (record id = 8444) being an NP so we dropped it\nmrn = 6351476 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record id = 8429) being an NP so we dropped it and reclassified it to one sample post\nmrn = 6891929 had 3 samples, classified as multiple sample post. 2 of the samples was BAL with 1 sample (record id = 8474) being an NP so we dropped it\nmrn = 7149759 had 2 samples, classified as multiple sample post. 1 of the samples was NP with 1 sample (record id = 32689) being a BAL so we dropped it and reclassified it to one sample post\nmrn = 7261558 had 2 samples, classified as two sample paired. 1 of the samples was RNA with 1 sample (record id = 8482) being a BAL so we dropped it and reclassified it to one sample post\nmrn = 7435740 had 7 samples, classified as multiple sample post. 4 of the samples was BAL with 1 sample (record id = 1

In [30]:
for label in rdv_hosp['sample_type'].unique():
    stdf = rdv_hosp[rdv_hosp['sample_type']==label]
    for mrn in stdf['mrn'].unique():
        print(stdf[stdf['mrn']==mrn].loc[:,['sample_date','rdvdate','ca_hosp_admit_dt','daysdiff_sample_rdv','sample_type','Sample Type (when received) ']])
        
        
        
        

  sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
0  2021-05-06 2021-05-04       2021-05-04                  2.0   
1  2021-05-12 2021-05-04       2021-05-04                  8.0   

            sample_type Sample Type (when received)   
0  multiple_sample_post                     VTM (NP)  
1  multiple_sample_post                     VTM (NP)  
  sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
5  2022-03-18 2022-03-16       2022-03-16                  2.0   
6  2022-03-24 2022-03-16       2022-03-16                  8.0   
7  2022-04-04 2022-03-16       2022-03-16                 19.0   

            sample_type Sample Type (when received)   
5  multiple_sample_post                     VTM (NP)  
6  multiple_sample_post                     VTM (NP)  
7  multiple_sample_post                     VTM (NP)  
   sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
19  2022-01-20 2022-01-05       2022-03-28                 15.0   
20  2022-01-27 20

83  one_sample_post                          BAL  
   sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
84  2023-01-28 2023-01-13       2023-01-13                 15.0   

        sample_type Sample Type (when received)   
84  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
100  2021-09-03 2021-08-26       2021-08-26                  8.0   

         sample_type Sample Type (when received)   
100  one_sample_post                          BAL  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
103  2021-12-29 2021-12-27       2021-12-27                  2.0   

         sample_type Sample Type (when received)   
103  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
106  2022-01-17 2022-01-16       2022-01-15                  1.0   

         sample_type Sample Type (when received)   
106  one_sample_post                     

301  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
306  2021-11-11 2021-11-09       2021-11-09                  2.0   

         sample_type Sample Type (when received)   
306  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
309  2021-12-09 2021-12-03       2021-12-07                  6.0   

         sample_type Sample Type (when received)   
309  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
310  2020-11-08 2020-10-21       2020-10-19                 18.0   

         sample_type Sample Type (when received)   
310  one_sample_post                     VTM (NP)  
    sample_date    rdvdate ca_hosp_admit_dt  daysdiff_sample_rdv  \
311  2021-01-14 2021-01-06       2021-01-13                  8.0   

         sample_type Sample Type (when received)   
311  one_sample_post                

In [31]:
len(rdv_hosp['mrn'].unique())

192

In [32]:
len(nordv_hosp['mrn'].unique())

193

In [33]:
for label in rdv_hosp['sample_type'].unique():
    print('###', label)
    print('unique mrn', len(rdv_hosp[rdv_hosp['sample_type']==label]['mrn'].unique()))
    print('unique samples', rdv_hosp[rdv_hosp['sample_type']==label].shape[0])

print('############################')
for label in nordv_hosp['sample_type'].unique():
    print('###',label)
    print('unique mrn', len(nordv_hosp[nordv_hosp['sample_type']==label]['mrn'].unique()))
    print('unique samples', nordv_hosp[nordv_hosp['sample_type']==label].shape[0])


### multiple_sample_post
unique mrn 37
unique samples 95
### one_sample_post
unique mrn 106
unique samples 106
### two_sample_paired
unique mrn 33
unique samples 66
### three_plus_paired
unique mrn 16
unique samples 79
############################
### one_sample_post
unique mrn 128
unique samples 128
### multiple_sample_post
unique mrn 42
unique samples 110
### three_plus_paired
unique mrn 6
unique samples 22
### two_sample_paired
unique mrn 17
unique samples 34


In [34]:
rdv_hosp_clinical = pd.merge(rdv_hosp[rdv_hosp['daysdiff_rdv_hosp']<0], clinicaldata, on = 'record_id')

In [35]:
clinicaldata['ca_hosp_admit_dt'] = pd.to_datetime(clinicaldata['ca_hosp_admit_dt'])

In [36]:
rdv_hosp[rdv_hosp['daysdiff_rdv_hosp']<0].loc[:,['sample_date','rdvdate','ca_hosp_admit_dt','daysdiff_rdv_hosp','mrn']].sort_values(by = 'mrn')

,sample_date,rdvdate,ca_hosp_admit_dt,daysdiff_rdv_hosp,mrn
12,2020-12-21,2020-11-26,2020-12-15,-19.0,1442906
19,2022-01-20,2022-01-05,2022-03-28,-82.0,1487858
20,2022-01-27,2022-01-05,2022-03-28,-82.0,1487858
31,2021-01-19,2020-11-28,2020-12-23,-9.0,5072963
29,2021-01-06,2020-11-28,2020-12-07,-9.0,5072963
28,2021-01-04,2020-11-28,2020-12-07,-9.0,5072963
27,2020-11-28,2020-11-28,2020-11-27,-9.0,5072963
30,2021-01-26,2020-11-28,2020-11-27,-9.0,5072963
25,2021-02-17,2020-11-28,2021-04-28,-9.0,5072963
24,2021-02-17,2020-11-28,2021-04-28,-9.0,5072963


In [37]:
#reconciling conflicting/missing hospitalization dates with earliest date recorded
for mrn in rdv_hosp[rdv_hosp['daysdiff_rdv_hosp']<0]['mrn'].unique():
    print('#########')
    idx = rdv_hosp[rdv_hosp['mrn']==mrn].index
    hospdate = pd.to_datetime(rdv_hosp[rdv_hosp['mrn']==mrn]['ca_hosp_admit_dt'].iloc[0])
    new_hospdate = pd.to_datetime(clinicaldata[clinicaldata['ca_mrn']==mrn]['ca_hosp_admit_dt'].sort_values().iloc[0])
    new_icudate = pd.to_datetime(clinicaldata[clinicaldata['ca_mrn']==mrn]['ca_icu_admit_dt'].sort_values().iloc[0])
    earliest_date = pd.Series([hospdate, new_hospdate, new_icudate]).sort_values().iloc[0]
    rdv_hosp.at[idx,'ca_hosp_admit_dt'] = earliest_date



#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########
#########


In [38]:
#days calculated all relative to first rdv_hosp or first RDV date
for mrn in rdv_hosp['mrn'].unique():
    pt = rdv_hosp[rdv_hosp['mrn']==mrn]
    rdv_hospdate = pt.sort_values(by = 'ca_hosp_admit_dt')['ca_hosp_admit_dt'].iloc[0] #first rdv_hosp date
    rdvdate = pt.sort_values(by = 'rdvdate')['rdvdate'].iloc[0] #first rdv date
    daysdiff_hosp = [x.days for x in (pt['sample_date'] - rdv_hospdate)]
    rdv_hosp.at[pt.index, 'daysdiff_sample_hosp'] = daysdiff_hosp

    daysdiffrdv_hosp = [x.days for x in (pt['rdvdate'] - rdv_hospdate)]
    rdv_hosp.at[pt.index, 'daysdiff_rdv_hosp'] = daysdiffrdv_hosp

    daysdiffrdv = [x.days for x in (pt['sample_date'] - rdvdate)]
    rdv_hosp.at[pt.index, 'daysdiff_sample_rdv'] = daysdiffrdv

rdv_hosp['ca_hosp_admit_dt'] = pd.to_datetime(rdv_hosp['ca_hosp_admit_dt'])

In [39]:
rdv_hosp[rdv_hosp['daysdiff_rdv_hosp']<0].loc[:,['sample_date','rdvdate','ca_hosp_admit_dt','daysdiff_rdv_hosp','mrn']].sort_values(by = 'mrn')

,sample_date,rdvdate,ca_hosp_admit_dt,daysdiff_rdv_hosp,mrn


In [40]:
'''
5825135 #one sample is is np and one is bal and both had own admit date so we dropped the later one, manually changed record id 8808 to 7/2/2021 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047
10589511 #only has one hosp date and its 9/30 but Egon said its 9/29 so will manually change this IN THE DATASET RDVSEQ_DATA_2023-08-18_1047
111011329869 #only has date on 6/4 but Egon said there are 4/23, 5/16, 5/31, manually change to 5/16 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047
111014419263 #only has 6/24 but Egon said its 5/15 or 5/12, manually change to 5/15 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047
'''

'\n5825135 #one sample is is np and one is bal and both had own admit date so we dropped the later one, manually changed record id 8808 to 7/2/2021 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047\n10589511 #only has one hosp date and its 9/30 but Egon said its 9/29 so will manually change this IN THE DATASET RDVSEQ_DATA_2023-08-18_1047\n111011329869 #only has date on 6/4 but Egon said there are 4/23, 5/16, 5/31, manually change to 5/16 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047\n111014419263 #only has 6/24 but Egon said its 5/15 or 5/12, manually change to 5/15 IN THE DATASET RDVSEQ_DATA_2023-08-18_1047\n'

In [41]:
# rdv_hosp.to_csv('/Users/tlh4362/Desktop/rdv_hosp.csv')

In [42]:
#labeling samples as "pre" or "post"
for mrn in rdv_hosp['mrn'].unique():
    pt = rdv_hosp[rdv_hosp['mrn']==mrn]
    
    if 'post' in pt['sample_type'].unique()[0]:
        rdv_hosp.at[pt.index,'RDV'] = 'post'
    else:
        preidx = pt[pt['daysdiff_sample_rdv']<=0].index
        rdv_hosp.at[preidx, 'RDV'] = 'pre'
        postidx = pt[pt['daysdiff_sample_rdv']>0].index
        rdv_hosp.at[postidx, 'RDV'] = 'post'


In [43]:
#labeling samples as "pre" or "post"

for mrn in nordv_hosp['mrn'].unique():
    pt = nordv_hosp[nordv_hosp['mrn']==mrn]
    
    if 'post' in pt['sample_type'].unique()[0]:
        nordv_hosp.at[pt.index,'Hosp'] = 'post'
    else:
        preidx = pt[pt['daysdiff_sample_hosp']<=0].index
        nordv_hosp.at[preidx, 'Hosp'] = 'pre'
        postidx = pt[pt['daysdiff_sample_hosp']>0].index
        nordv_hosp.at[postidx, 'Hosp'] = 'post'


In [44]:
# sequences that don't match up were dropped
rdv_hosp = rdv_hosp.drop(283, axis = 0) #omicron sequence amongst 2 deltas

rdv_hosp.at[[282,284],'sample_type'] = 'two_sample_paired'

rdv_hosp = rdv_hosp.drop(337, axis = 0) #omicron sequence amongst 4 deltas

In [45]:
nordv_hosp = nordv_hosp.drop(184, axis = 0) #1 omicron amongst deltas 

In [46]:
# rdv_hosp.to_csv('/Users/tlh4362/Desktop/rdv_hosp.csv')
# nordv_hosp.to_csv('/Users/tlh4362/Desktop/control_hosp.csv')

In [47]:
#missing CT values, manual look up
idx = rdv_hosp[rdv_hosp['record_id'] == 41970].index
rdv_hosp.at[idx,'ct_n1'] = 26.0685

idx = rdv_hosp[rdv_hosp['record_id'] == 42664].index
rdv_hosp.at[idx,'ct_n1'] = 22.867

idx = rdv_hosp[rdv_hosp['record_id'] == 41555].index
rdv_hosp.at[idx,'ct_n1'] = 30.4605


idx = nordv_hosp[nordv_hosp['record_id'] == 40460].index
nordv_hosp.at[idx,'ct_n1'] = 18.9095

idx = nordv_hosp[nordv_hosp['record_id'] == 42005].index
nordv_hosp.at[idx,'ct_n1'] = 28.1068

idx = nordv_hosp[nordv_hosp['record_id'] == 42147].index
nordv_hosp.at[idx,'ct_n1'] = 21.5245